# **Fine-Tuning Biomedical Models on Text Classification Task With HuggingFace Transformers and PyTorch XLA**

First, you need to enable the TPU by going to "change runtime" -> TPU. Google Colab Pro give you more memory for TPU (35GB) which will be helpful with PyTorch XLA.

In [ ]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

     |████████████████████████████████| 149.9 MB 46 kB/s 
     |████████████████████████████████| 57 kB 2.9 MB/s 
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.278 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.8.0 which is incompatible.


This Google Colab notebook aims to demonstrate the use of TPU with with PyTorch and HuggingFace Transformers library. PyTorch XLA https://github.com/pytorch/xla allows you to use PyTorch on TPU unit. Please note that this notebook is experimental, and we strongly advise you to use the original implementation of ELECTRA and ALBERT to reproduce any results. We created this notebook to help reseachers with limited resources to work with larger models such as ELECTRA-large and ALBERTxxlarge.

In [ ]:
!git clone https://github.com/huggingface/transformers
!pip3 install -e transformers
!pip3 install -r transformers/examples/pytorch/text-classification/requirements.txt

Cloning into 'transformers'...
remote: Enumerating objects: 83635, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 83635 (delta 10), reused 10 (delta 2), pack-reused 83607
Receiving objects: 100% (83635/83635), 66.73 MiB | 34.34 MiB/s, done.
Resolving deltas: 100% (60034/60034), done.
Obtaining file:///content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 50 kB 3.0 MB/s 
     |████████████████████████████████| 636 kB 8.6 MB/s 
     |████████████████████████████████| 3.3 MB 61.9 MB/s 
     |████████████████████████████████| 895 kB 67.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Running setup.py develop for transformers
     |████████████████████████████████| 55 kB 2.2 MB/s 
    

In [ ]:
!pip install -r /content/transformers/examples/pytorch/_tests_requirements.txt

     |████████████████████████████████| 43 kB 1.1 MB/s 
     |████████████████████████████████| 90 kB 4.7 MB/s 
     |████████████████████████████████| 8.4 MB 71.8 MB/s 
     |████████████████████████████████| 8.0 MB 40.1 MB/s 
     |████████████████████████████████| 363 kB 76.6 MB/s 
     |████████████████████████████████| 87 kB 6.0 MB/s 
     |████████████████████████████████| 170 kB 71.5 MB/s 
     |████████████████████████████████| 111 kB 63.4 MB/s 
     |████████████████████████████████| 75 kB 4.1 MB/s 
     |████████████████████████████████| 4.3 MB 62.3 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 123 kB 69.7 MB/s 
     |████████████████████████████████| 787 kB 65.4 MB/s 
     |████████████████████████████████| 370 kB 74.2 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=16257fb334fafb816aa846b2071a46c64c3614412bda41f2d6280b73eea3b085
  Stored in directory: /root/.cache/pip/wheels

In [ ]:
!wget https://microsoft.github.io/BLURB/sample_code/data_generation.tar.gz
!tar -xvf data_generation.tar.gz
import nltk
nltk.download('punkt')

--2021-09-07 16:38:15--  https://microsoft.github.io/BLURB/sample_code/data_generation.tar.gz
Resolving microsoft.github.io (microsoft.github.io)... 185.199.109.153, 185.199.110.153, 185.199.111.153, ...
Connecting to microsoft.github.io (microsoft.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54142 (53K) [application/gzip]
Saving to: ‘data_generation.tar.gz’

data_generation.tar 100%[===================>]  52.87K  --.-KB/s    in 0.01s   

2021-09-07 16:38:15 (4.68 MB/s) - ‘data_generation.tar.gz’ saved [54142/54142]

data_generation/
data_generation/download_BLURB_data.sh
data_generation/indexing/
data_generation/indexing/BioASQ/
data_generation/indexing/BioASQ/dev_id.tsv
data_generation/indexing/BioASQ/test_id.tsv
data_generation/indexing/BioASQ/train_id.tsv
data_generation/indexing/DDI/
data_generation/indexing/DDI/dev_files.tsv
data_generation/indexing/DDI/test_files.tsv
data_generation/indexing/DDI/train_files.tsv
data_generat

True

In [ ]:
%%shell
pip3 install -r /content/data_generation/requirements.txt
cd data_generation
sh download_BLURB_data.sh
wget https://biocreative.bioinformatics.udel.edu/media/store/files/2017/ChemProt_Corpus.zip
mv ChemProt_Corpus.zip raw_data
sh preprocess_BLURB_data.sh

     |████████████████████████████████| 1.4 MB 5.2 MB/s 
     |████████████████████████████████| 5.5 MB 23.1 MB/s 
     |████████████████████████████████| 5.5 MB 62.3 MB/s 
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434690 sha256=cc0610495270887e7fae8d2b378ad4705231d749b8651ad29d69e804bb023346
  Stored in directory: /root/.cache/pip/wheels/45/6c/46/a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
  Created wheel for python-docx: filename=python_docx-0.8.10-py3-none-any.whl size=184397 sha256=1d224c7517f013635eb881e38a8b36066223fa741693452d12b126e0ee5e411a
  Stored in directory: /root/.cache/pip/wheels/75/c6/69/05491f32dc052cd70476b65f5bf7082a9b274045f6b001b821
Successfully built nltk python-docx
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
  Attempting uninstall: zipp
    Found existing installation: zipp 3.5.0
    Uninstalling zipp-3.5.0:
      Successfully unin

In [ ]:
!mkdir data
!cp -r /content/data_generation/data/DDI data
!cp -r /content/data_generation/data/GAD data
!cp -r /content/data_generation/data/chemprot data

In [ ]:
import pandas as pd
import os
import csv
import glob

def tsv_to_csv(path,skip_first_row):
  for file in glob.glob(path):
    print(file)
    csv_table=pd.read_csv(file,header=None,skiprows=skip_first_row, delimiter="\t", quoting=3, encoding='utf-8',names=["index", "sentence1", "label"],usecols=["sentence1","label"])
    csv_table.to_csv(os.path.splitext(file)[0]+".csv",index=False)

tsv_to_csv('/content/data/chemprot/*.tsv',0)
tsv_to_csv('/content/data/GAD/*.tsv',1)  ## we skip first row since GAD tsv files has header row
tsv_to_csv('/content/data/DDI/*.tsv',0)

/content/data/chemprot/test.tsv
/content/data/chemprot/train.tsv
/content/data/chemprot/dev.tsv
/content/data/GAD/test.tsv
/content/data/GAD/train.tsv
/content/data/GAD/dev.tsv
/content/data/DDI/test.tsv
/content/data/DDI/train.tsv
/content/data/DDI/dev.tsv


**Evaluation Classes**

ChemProt dataset has six classes, including: {FALSE CPR:4 CPR:5 CPR:9 CPR:6 CPR:3}. However, as it is common practice in the literature, we evaluate only five classes {CPR:4 CPR:5 CPR:9 CPR:6 CPR:3}. As a result, do not use "do_eval" with chemprot as it will show you the eval accuracy with the "False" class, increasing the accuracy score. Instead, we will predict using "do_predict" then evaluate using sklearn code. This is also applicable to other datasets in the biomedical domain, so make sure you follow the standard practice in choosing which class is included in the evaluation. 

**PyTorch XLA**

We will use the PyTorch XLA function by adding this line to the PyTorch code: 

` python3 /content/transformers/examples/pytorch/xla_spawn.py --num_cores 8` 

run_glue.py code is designed to work with GLUE Benchmark and also with any text classification task .

Please note that if you run the code for the first time, pre-processing the data and TPU graph will be slow, but as it stores it on the cache, your next run with the same batch size will be much faster. Every time you change the batch size, the pre-processing will become slow. This process may take minutes for a larger model, so be patient. Also, when you choose a batch size of 16, your batch size will be 16*8=128 since we use TPUv3-8 thas has eight cores. This notebook is experimental, and you should use different hyperparameters than our paper since the implementation is different. We choose test_file to be dev.tsv since we need to predict the dev file. You need validation_file, or otherwise, the code will throw an error.




In [ ]:
import os
model= "sultan/BioM-ELECTRA-Large-Discriminator" #@param ["sultan/BioM-ELECTRA-Base-Discriminator", "sultan/BioM-ELECTRA-Large-Discriminator","sultan/BioM-ALBERT-xxlarge","sultan/BioM-ALBERT-xxlarge-PMC","dmis-lab/biobert-base-cased-v1.1","emilyalsentzer/Bio_ClinicalBERT","microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract","microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext","allenai/scibert_scivocab_uncased","allenai/scibert_scivocab_cased","BioM-BERT-PyTorch"]
task= "chemprot" #@param ["chemprot", "GAD", "DDI"]
batch_size = 8 #@param {type:"slider", min:4, max:128, step:4}
learning_rate = "5e-5"#@param ["1e-4", "3e-4", "1e-5","3e-5","5e-5","7e-5"]
epochs_num = 5 #@param {type:"slider", min:1, max:50, step:1}
max_seq_length= "256" #@param ["128", "256", "384","512"]
os.environ['batch_size'] = str(batch_size)
os.environ['learning_rate'] = str(learning_rate)
os.environ['epochs_num'] = str(epochs_num)
os.environ['task'] = str(task)
os.environ['model'] = str(model)
os.environ['max_seq_length'] = str(max_seq_length)
!python /content/transformers/examples/pytorch/xla_spawn.py --num_cores 8 transformers/examples/pytorch/text-classification/run_glue.py --model_name_or_path $model \
--train_file data/$task/train.csv \
--validation_file data/$task/dev.csv \
--test_file data/$task/dev.csv \
--output_dir output_dir/$task \
--overwrite_cache \
--overwrite_output_dir \
--logging_steps 1000000 \
--max_seq_length $max_seq_length \
--per_device_train_batch_size $batch_size \
--learning_rate $learning_rate \
--warmup_ratio 0.1 \
--num_train_epochs $epochs_num \
--save_steps 50000 \
--do_train \
--do_predict

09/07/2021 17:19:07 - WARNING - run_glue - Process rank: -1, device: xla:1, n_gpu: 0distributed training: False, 16-bits training: False
09/07/2021 17:19:07 - INFO - run_glue - Training/evaluation parameters TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=output_dir/c

Here, we are evaluating on five classes only. If your task to evaluate on all classes, then remove "labels=" option.

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score
predictions=pd.read_csv('/content/output_dir/chemprot/predict_results_None.txt',sep='\t',usecols=[1],skiprows=1,header=None)
Actuals=pd.read_csv('data/chemprot/dev.csv',sep=',',usecols=[1],skiprows=1,header=None)
f1_score(Actuals, predictions, average='micro',labels=['CPR:6','CPR:4','CPR:3','CPR:5','CPR:9'])

0.793933265925177

After finding the best hyperparameters on the Dev dataset, predict using those best hyperparameters. Then, choose --model to the one we generated to output dir (output_dir/$task ). We added the validation_file and train_file below so the code will not throw an error.

In [ ]:
!python /content/transformers/examples/pytorch/xla_spawn.py --num_cores 8 transformers/examples/pytorch/text-classification/run_glue.py --model_name_or_path output_dir/$task \
--train_file data/$task/train.csv \
--validation_file data/$task/dev.csv \
--test_file data/$task/test.csv \
--output_dir temp \
--overwrite_cache \
--overwrite_output_dir \
--max_seq_length $max_seq_length \
--do_predict

09/07/2021 17:52:14 - WARNING - run_glue - Process rank: -1, device: xla:1, n_gpu: 0distributed training: False, 16-bits training: False
09/07/2021 17:52:14 - INFO - run_glue - Training/evaluation parameters TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=True,
do_train=False,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=temp/runs/S

In [17]:
predictions=pd.read_csv('/content/temp/predict_results_None.txt',sep='\t',usecols=[1],skiprows=1,header=None)
Actuals=pd.read_csv('data/chemprot/test.csv',sep=',',usecols=[1],skiprows=1,header=None)
f1_score(Actuals, predictions, average='micro',labels=['CPR:6','CPR:4','CPR:3','CPR:5','CPR:9'])

0.7857769973661106

You need to find the best hyperparameters and not relying on hyperparameters that we published in our paper since the implementation is different. To reproduce our results with the same hyperparameters in our paper, use the other example that we build with TPU and Google Bucket.